In [1]:
import os
from timeit import default_timer as timer
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')

In [1]:
def get_file_list(dir_path):
    return os.listdir(dir_path)

class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    
    
class categorise_x(): #flexible number of categories
    
    def __init__(self,x):
        self.threshold=[]
        self.percentiles=[]
        self.num=x
        for i in range(1,x):
            self.percentiles.append(i*100/x)        
            
    def fit(self,array):
        
        positive=array[array>0]
        negative=array[array<=0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
        
    def return_quartile(self,array):
        temp=[]
        for num in array:
            if num<0:
                counter=0
                for i in self.threshold[0]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+1)
            else:
                counter=0
                for i in self.threshold[1]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+self.num+1)
        return np.asarray(temp)
    
class cross():
    def __init__(self):
        self.time_last_cross=0
        self.current_sign=True
        self.last_time=datetime(1900, 1, 1, 8, 59)
    def get_time(self,time,price):
        if (time-self.last_time)>timedelta(minutes=1):
            self.last_time=time
            self.time_last_cross=time
            return 0
        self.last_time=time
        if (price>0) and self.current_sign : #if price positive and current trend is also positive
            return (time-self.time_last_cross).total_seconds()
        elif (price<0) and (not self.current_sign): #if price negative and current trend is negative
            return (time-self.time_last_cross).total_seconds()
        else: #if price positive, trend negative or price negative, trend positive
            self.time_last_cross=time
            self.current_sign=(price>0)
            return 0

In [106]:
def calc_smart_price(bid_price,bid_vol,ask_price,ask_vol):
    bid_price=bid_price.copy()
    ask_price=ask_price.copy()
    bid_vol=bid_vol.copy()
    ask_vol=ask_vol.copy()    
    
    if len(bid_price)!= len(bid_vol) or len(bid_vol)!=len(ask_price) or len(ask_price)!=len(ask_vol) or len(ask_vol)!=len(bid_price):
        return
    
    #to combat the limit up event, where price is set to 0. 
    rows=(bid_price==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        bid_price[rows]=ask_price[rows] #for that row, assign ask price to it
    rows=(ask_price==0)
 #do the same for ask price
    if (np.any(rows)):
        ask_price[rows]=bid_price[rows]
    smart_price=(bid_price*ask_vol+ask_price*bid_vol)/(ask_vol+bid_vol)    
    return smart_price

def calc_present_vol(vol):
    vol=vol.copy()
    vol=np.diff(vol)
    vol=np.insert(vol,0,0)
    return vol

def calc_future_price(time_array, price_array,time_ahead=30):
    if len(time_array)!=len(price_array):
        return
    time_array=time_array.copy()
    price_array=price_array.copy()
    future_price=[]
    length=len(time_array)
    for i in range(len(time_array)):
        current_time=time_array[i]+timedelta(seconds=time_ahead)
        
        j=0 #could alternatively use 30 x 3 then search forward and backward
        
        #search forwards
        while((i+j)<length and current_time>time_array[i+j]):
            j+=1
        if (i+j)<length:
            #if index is in the dataframe
            future_price.append(price_array[(i+j)]) 
        else:
            #price ahead does not exist
            future_price.append(np.nan) 
    future_price=np.asarray(future_price)
    return future_price

def calc_edge(future_price,current_price):
    future_price=future_price.copy()
    current_price=current_price.copy()
    return (future_price-current_price)

def set_index(time_array):
    time_array=time_array.copy().astype(int)
    new_index=[]
    for j in range(len(time_array)):
        i=str(time_array[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    new_index=np.asarray(new_index)
    return new_index

def calc_sma_fast(price_array,time_array,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    price_array=price_array.copy()
    time_array=time_array.copy()
    if len(price_array)!=len(time_array):
        return
    sma_values=[] 
    smart_sum=np.cumsum(price_array) 
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=time_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                sma=(smart_sum[i]-smart_sum[i-j])/j
                sma_values.append(sma)                
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    sma=(smart_sum[i]-smart_sum[i-j])/j
                    sma_values.append(sma)   
                    
                else:
                    sma_values.append(price_array[i])
                    
        else: #starting point is at 0
            
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)                       

    sma_values=np.asarray(sma_values)
    return sma_values


def calc_past_vol(vol_array,time_array,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    vol_array=vol_array.copy()
    time_array=time_array.copy()
    if len(vol_array)!=len(time_array):
        return
    vol_values=[] 
    vol_sum=np.cumsum(vol_array) 
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=vol_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                vol=(vol_sum[i]-vol_sum[i-j])
                vol_values.append(vol)                
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    vol=(smart_sum[i]-smart_sum[i-j])
                    vol_values.append(vol)   
                    
                else: #if j=0, means i-j=i and past volume is 0
                    vol_values.append(0)
                    
        else: #starting point is at 0
            
            vol=vol_sum[i]
            vol_values.append(vol)                       

    vol_values=np.asarray(vol_values)
    return vol_values

def ewma_array(array,alpha=0.05,confidence=0.99,tolerance=1e-07):
    final=0
    array=alpha*array
    
    if len(array)==1:
        return(array[0])
    
    #array needs to have at least n samples to achieve confidence
    n=np.log((1-confidence))/np.log((1-alpha))
    
    for i in range(1,len(array)+1):
        current_day_ewma=array[-i]*((1-alpha)**(i-1))
        
        if (current_day_ewma<tolerance):
            #print('break')
            break
        final+=current_day_ewma 
        
    if len(array)<=n:
        final=final/(1-(1-alpha)**len(array))
        #print('adjusted',n)
    return final 

def calc_ewma(price_array,time_array,duration=1,alpha=0.1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    price_array=price_array.copy()
    time_array=time_array.copy()
    
    if len(price_array)!=len(time_array):
        return
    if len(price_array)==1:
        return price_array
    
    ewma_values=[] 
    
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=time_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                ewma=ewma_array(price_array[i-j:i+1])
                ewma_values.append(ewma)       
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    ewma=ewma_array(price_array[i-j:i+1])
                    ewma_values.append(ewma)   
                    
                else:
                    ewma_values.append(price_array[i])
                    
        else: #starting point is at 0
            
            ewma=ewma_array(price_array[:i+1])
            ewma_values.append(ewma)                

    ewma_values=np.asarray(ewma_values)
    return ewma_values

def last_cross(dataset,price_array,time_array):
    price_array=price_array.copy()
    time_array=time_array.copy()
    last_cross=cross()
    timings=[]
    for i in range(len(price-array)):
        timings.append(last_cross.get_time(time_array[i],price_array[i]))
    timings=np.asarray(timings)
    return timings

def get_case(array):
    current=1
    previous=0
    temp=[]
    for i in array:
        if i==current:
            if i>previous:
                temp.append(1)
            else:
                temp.append(0)
        elif i>current:
            previous=current
            current=i
            temp.append(1)
        else:
            previous=current
            current=i
            temp.append(0)
    return np.asarray(temp)


def binaryToDecimal(binary): 
      
    binary1 = binary 
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return(decimal) 
    
def get_case_n(array,n=2):
    temp=[]
    if n==1:
        return("error")
    previous=np.zeros(n+1)
    for i in array:
        #print(i,previous)
        final=''
        if i==int(previous[-1]):
            before=10
            for j in previous:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
        else:
            before=10
            for j in previous[1:]:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
            if i>before:
                final+='1'
            else:
                final+='0'
        
            previous=np.delete(previous,0)
            previous=np.insert(previous,n,i)
        #print(final)
        temp.append(binaryToDecimal(int(final)))
    return np.asarray(temp)    

def process(dataset,sma_duration=1,**args):
    data=dataset.copy().loc[:,['Time','BidPrice1','AskPrice1','BidVol1','AskVol1']].values
    
    #calc smart price 
    smart_price=calc_smart_price(data[:,1],data[:,3],data[:,2],data[:,4])
    
    #set index to datetime format
    time_array=set_index(data[:,0]) 
    
    #calculate future price
    future_price=calc_future_price(time_array,smart_price)
    
    #calculate edge 
    edge=calc_edge(future_price=future_price,current_price=smart_price)
    
    #calculate SMA
    sma=calc_sma_fast(time_array=time_array,price_array=smart_price)
    sma=smart_price-sma
    
    smart_price=np.expand_dims(smart_price,axis=1)
    time_array=np.expand_dims(time_array,axis=1)
    future_price=np.expand_dims(future_price,axis=1)
    edge=np.expand_dims(edge,axis=1)
    sma=np.expand_dims(sma,axis=1)
    
    data=np.concatenate((time_array,smart_price,future_price,edge,sma),axis=1)
    
    return data

In [111]:
#path="D:\\Dropbox\\TChina\\Futures\\data\\day\\TA\\"
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
def get_all_technicals(path,process_function):
    df_list=[]
    name_list=[]
    dir_list=os.listdir(path)
    dir_list.sort()
    for file in dir_list:
        if file[-3:]=='csv':
            name_list.append(file)
            df_list.append(process_function(pd.read_csv(path+file),sma_duration=1))
            #print(file)
    print('complete')
    return df_list,name_list
            

In [112]:
df_list,name_list=get_all_technicals(path,process)

2019.01.02.csv
2019.01.03.csv
2019.01.04.csv
2019.01.07.csv
2019.01.08.csv
2019.01.09.csv
2019.01.10.csv
2019.01.11.csv
2019.01.14.csv
2019.01.15.csv
2019.01.16.csv
2019.01.17.csv
2019.01.18.csv
2019.01.21.csv
2019.01.22.csv
2019.01.23.csv
2019.01.24.csv
2019.01.25.csv
2019.01.28.csv
2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.04.csv
2019.04.08.csv
2019.04.09.csv
2019.04.10.csv
2019.04.11.csv
2019.04.12